# **ZincSight**: A rapid and precise tool for large-scale zinc ion location prediction in proteins

If the runtime crashes during ZincSight execution, click `Runtime` → `Disconnect and delete runtime` and then start over. If the issue presists, contact mechtinger1@mail.tau.ac.il and include the protein IDs/structures that you used.






In [ ]:
#@title Click '▶' for Initialization {display-mode: "form"}
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
#@title Query for ZincSight {display-mode: "form"}
#@markdown  Enter your protein(s) in the field below or upload PDB or MMCIF structures. Then hit `Runtime` → `Run all`.
# @markdown - Query IDs can be PDB, UniProt, AlphaFold, or ESM identifiers, for example: `8QEP, P0A6G5, AF-A0A068N621-F1-v4, MGYP002718891411`.
# @markdown -Alternatively, leave the field empty in order to upload your own structure files.
#@markdown  <p></p>
#@markdown  <p></p>
structure_ids_for_download = '8QEP, P0A6G5, AF-A0A068N621-F1-v4, MGYP002718891411' # @param {type:"string"}


upload_structures = False

include_histidine_rotamers = True # @param {type: "boolean"}
# If you do not want to include rotamer predictions, uncheck it.


if structure_ids_for_download.strip() == '':
  !mkdir -p /content/query_structures
  %cd /content/query_structures
  from google.colab import files
  uploaded = files.upload()
  %cd /content

In [ ]:
%%capture captured_output
#@title Excute ZincSight {display-mode: "form"}
!pip install biopython==1.79
!conda install pymol-open-source=3.0.0 scipy matplotlib psycopg2-binary scikit-learn python-dotenv joblib aria2p -c conda-forge

!sudo apt install pigz

# Install Essential System Packages and GPG Tools
!sudo apt install dirmngr ca-certificates software-properties-common gnupg gnupg2 apt-transport-https curl -y

# Install PostgresSQL
!curl -fSsL https://www.postgresql.org/media/keys/ACCC4CF8.asc | gpg --dearmor | sudo tee /usr/share/keyrings/postgresql.gpg > /dev/null
!echo 'deb [arch=amd64,arm64,ppc64el signed-by=/usr/share/keyrings/postgresql.gpg] http://apt.postgresql.org/pub/repos/apt/ jammy-pgdg main' | sudo tee /etc/apt/sources.list.d/pgdg.list
!sudo apt update
!sudo apt install postgresql-client-15 postgresql-15 -y
!sudo service postgresql start

# Create a new PostgreSQL database if it does not already exist
!sudo -u postgres psql -tc "SELECT 1 FROM pg_database WHERE datname = 'zincsight_pipeline_db';" | grep -q 1 || sudo -u postgres psql -c "CREATE DATABASE zincsight_pipeline_db;"
!sudo -u postgres psql -c "GRANT ALL PRIVILEGES ON DATABASE zincsight_pipeline_db TO postgres;"
!sudo -u postgres psql -c "ALTER USER postgres WITH PASSWORD 'postgres';"

# Clone the GitHub repository and change into the repo directory
!git clone https://github.com/MECHTI1/ZincSight.git
%cd ZincSight/

# Add essential SQL tables to the established database
!sudo -u postgres psql -d zincsight_pipeline_db -P pager=off -f "src/setup_pg_db_with_tables/PostgreSQL_4_necessary_tables.sql"


import os
print("Current Path:", os.getcwd()) # Debugging: Print current working directory

import sys
sys.path.append('/content/ZincSight')
from main_execute import execute_zincsight

# Define paths explicitly
query_structures_path = '/content/query_structures'  # Path to input structures
output_path = '/content/output'  # Path to save output

# Ensure paths exist
if not os.path.exists(query_structures_path):
    os.makedirs(query_structures_path)
if not os.path.exists(output_path):
    os.makedirs(output_path)  # Create output directory if it doesn't exist

# Call the function with explicit paths
output_compressed_file = execute_zincsight(
    include_rotamers,       # Boolean flag
    structure_ids_for_download,          # String of structure IDs
    query_structures_path,  # Input directory path
    output_path             # Output directory path
)

In [ ]:
#@title Download Result{display-mode: "form"}
from google.colab import files
files.download(output_compressed_file)